In [25]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
import random
from datetime import datetime, timedelta
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output


In [26]:
truck_types = [
    {
        'Truck_ID': 'T1',
        'Truck_Type': 'Multilift 4 Axles',
        'Category': ['Waste', 'Enrobé'],
        'Capabilities': ['Standard Access'],
        'Possible_Equipment': {
            'Benne 10 m³': 15,
            'Benne 20 m³': 18,
            'Thermos': 18,
            'Pont 13 m³': 15
        },
        'Available': True
    },
    {
        'Truck_ID': 'T2',
        'Truck_Type': 'Multilift 4 Axles',
        'Category': ['Waste', 'Enrobé'],
        'Capabilities': ['Standard Access'],
        'Possible_Equipment': {
            'Benne 10 m³': 15,
            'Benne 20 m³': 18,
            'Thermos': 18,
            'Pont 13 m³': 15
        },
        'Available': True
    },
    {
        'Truck_ID': 'T3',
        'Truck_Type': 'Multilift 4 Axles Tridem',
        'Category': ['Waste', 'Enrobé'],
        'Capabilities': ['Difficult Access'],
        'Possible_Equipment': {
            'Benne 10 m³': 15,
            'Benne 20 m³': 18,
            'Thermos': 18,
            'Pont 13 m³': 15
        },
        'Available': True
    },
    {
        'Truck_ID': 'T4',
        'Truck_Type': 'Camion-grue 4 Axles',
        'Category': ['Crane Services'],
        'Capabilities': ['Equipped with Crane'],
        'Possible_Equipment': {
            'Crane': 8.5,
            'Remorque Tandem': 8.5  # Optional additional equipment
        },
        'Available': True
    },
    {
        'Truck_ID': 'T5',
        'Truck_Type': 'Multilift 4 Axles',
        'Category': ['Waste'],
        'Capabilities': ['Standard Access'],
        'Possible_Equipment': {
            'Pont 13 m³': 15
        },
        'Available': True
    }
]

# Convert to DataFrame for better handling
trucks_df = pd.DataFrame(truck_types)
trucks_df


,Truck_ID,Truck_Type,Category,Capabilities,Possible_Equipment,Available
0,T1,Multilift 4 Axles,"[Waste, Enrobé]",[Standard Access],"{'Benne 10 m³': 15, 'Benne 20 m³': 18, 'Thermo...",True
1,T2,Multilift 4 Axles,"[Waste, Enrobé]",[Standard Access],"{'Benne 10 m³': 15, 'Benne 20 m³': 18, 'Thermo...",True
2,T3,Multilift 4 Axles Tridem,"[Waste, Enrobé]",[Difficult Access],"{'Benne 10 m³': 15, 'Benne 20 m³': 18, 'Thermo...",True
3,T4,Camion-grue 4 Axles,[Crane Services],[Equipped with Crane],"{'Crane': 8.5, 'Remorque Tandem': 8.5}",True
4,T5,Multilift 4 Axles,[Waste],[Standard Access],{'Pont 13 m³': 15},True


In [27]:
# Define client needs and corresponding equipment
client_needs = {
    'Waste': ['Benne 10 m³', 'Benne 20 m³', 'Pont 13 m³'],
    'Enrobé': ['Thermos'],
    'Crane Service': ['Crane']  # Initial assignment; Remorque Tandem handled separately
}

# Define time windows
time_windows = [
    ('Morning', '08:00', '12:00'),
    ('Afternoon', '13:00', '17:00'),
    ('All Day', '08:00', '17:00')
]


In [28]:
# Paris geographical boundaries
# Approximate bounding box for Paris
paris_bounds = {
    'lat_min': 48.815573,
    'lat_max': 48.902144,
    'lon_min': 2.224199,
    'lon_max': 2.469920
}


In [29]:
# Cell 6: Function to Generate Fake Client Data (Corrected)
def generate_clients(num_clients, waste_ratio=0.6, enrobe_ratio=0.3, crane_ratio=0.1,
                    difficult_access_ratio=0.2, time_window_distribution=None,
                    remorque_tandem_ratio=0.3):
    """
    Generates fake client data.

    Parameters:
    - remorque_tandem_ratio: Proportion of Crane Service tasks that require Remorque Tandem.
    """
    if time_window_distribution is None:
        time_window_distribution = {'Morning': 0.4, 'Afternoon': 0.4, 'All Day': 0.2}
    
    # Assign client needs based on ratios
    needs = np.random.choice(['Waste', 'Enrobé', 'Crane Service'],
                             size=num_clients,
                             p=[waste_ratio, enrobe_ratio, crane_ratio])
    
    # Assign access difficulty
    access_difficulty = np.random.choice(['Yes', 'No'],
                                        size=num_clients,
                                        p=[difficult_access_ratio, 1 - difficult_access_ratio])
    
    # Assign time windows based on distribution
    time_window_labels = list(time_window_distribution.keys())
    time_window_probs = list(time_window_distribution.values())
    time_window = np.random.choice(time_window_labels,
                                   size=num_clients,
                                   p=time_window_probs)
    
    # Generate random locations
    locations = generate_random_location(num_clients)
    
    # Assign volumes based on client needs
    volumes = []
    for need in needs:
        if need == 'Waste':
            volume = np.random.uniform(10, 20)  # in tonnes
        elif need == 'Enrobé':
            volume = np.random.uniform(13, 18)  # in tonnes (Thermos capacity)
        elif need == 'Crane Service':
            # Some Crane Services might require Remorque Tandem
            requires_tandem = np.random.rand() < remorque_tandem_ratio
            if requires_tandem:
                volume = 17  # For example, double capacity
            else:
                volume = 8.5  # Standard capacity
        volumes.append(round(volume, 2))
    
    clients_df = pd.DataFrame({
        'Client_ID': [f'C{str(i).zfill(3)}' for i in range(1, num_clients + 1)],
        'Need': needs,
        'Volume_t': volumes,
        'Access_Difficulty': access_difficulty,
        'Time_Window_Label': time_window,
        'Latitude': locations['Latitude'],
        'Longitude': locations['Longitude']
    })
    
    # Assign specific equipment based on Need and Access Difficulty
    equipment = []
    for index, row in clients_df.iterrows():
        if row['Need'] == 'Crane Service':
            if row['Volume_t'] > 8.5:
                # Requires Remorque Tandem in addition to Crane
                equipment.append(['Crane', 'Remorque Tandem'])
            else:
                # Only Crane is needed
                equipment.append(['Crane'])
        else:
            # For Waste and Enrobé, select equipment based on Access Difficulty
            possible_equipment = client_needs[row['Need']]
            if row['Access_Difficulty'] == 'Yes':
                # Prefer trucks with Difficult Access capability
                equipment.append([random.choice(['Benne 10 m³', 'Benne 20 m³', 'Pont 13 m³'])])
            else:
                equipment.append([random.choice(possible_equipment)])
    
    clients_df['Required_Equipment'] = equipment
    
    return clients_df



In [30]:
def generate_random_location(num_locations):
    latitudes = np.random.uniform(paris_bounds['lat_min'], paris_bounds['lat_max'], num_locations)
    longitudes = np.random.uniform(paris_bounds['lon_min'], paris_bounds['lon_max'], num_locations)
    return pd.DataFrame({
        'Latitude': latitudes,
        'Longitude': longitudes
    })


In [36]:
def generate_clients(num_clients, waste_ratio=0.6, enrobe_ratio=0.3, crane_ratio=0.1,
                    difficult_access_ratio=0.2, time_window_distribution=None,
                    remorque_tandem_ratio=0.3):
    """
    Generates fake client data.

    Parameters:
    - remorque_tandem_ratio: Proportion of Crane Service tasks that require Remorque Tandem.
    """
    if time_window_distribution is None:
        time_window_distribution = {'Morning': 0.4, 'Afternoon': 0.4, 'All Day': 0.2}
    
    # Assign client needs based on ratios
    needs = np.random.choice(['Waste', 'Enrobé', 'Crane Service'],
                             size=num_clients,
                             p=[waste_ratio, enrobe_ratio, crane_ratio])
    
    # Assign access difficulty
    access_difficulty = np.random.choice(['Yes', 'No'],
                                        size=num_clients,
                                        p=[difficult_access_ratio, 1 - difficult_access_ratio])
    
    # Assign time windows based on distribution
    time_window_labels = list(time_window_distribution.keys())
    time_window_probs = list(time_window_distribution.values())
    time_window = np.random.choice(time_window_labels,
                                   size=num_clients,
                                   p=time_window_probs)
    
    # Generate random locations
    locations = generate_random_location(num_clients)
    
    # Assign volumes based on client needs
    volumes_t = []
    volumes_kg = []
    for need in needs:
        if need == 'Waste':
            volume = np.random.uniform(10, 20)  # in tonnes
        elif need == 'Enrobé':
            volume = np.random.uniform(13, 18)  # in tonnes (Thermos capacity)
        elif need == 'Crane Service':
            # Some Crane Services might require Remorque Tandem
            requires_tandem = np.random.rand() < remorque_tandem_ratio
            if requires_tandem:
                volume = 17  # For example, double capacity
            else:
                volume = 8.5  # Standard capacity
        volumes_t.append(round(volume, 2))
        volumes_kg.append(round(volume * 1000, 2))  # Convert to kg
    
    clients_df = pd.DataFrame({
        'Client_ID': [f'C{str(i).zfill(3)}' for i in range(1, num_clients + 1)],
        'Need': needs,
        'Volume_t': volumes_t,
        'Volume_kg': volumes_kg,
        'Access_Difficulty': access_difficulty,
        'Time_Window_Label': time_window,
        'Latitude': locations['Latitude'],
        'Longitude': locations['Longitude']
    })
    
    # Assign specific equipment based on Need and Access Difficulty
    equipment = []
    for index, row in clients_df.iterrows():
        if row['Need'] == 'Crane Service':
            if row['Volume_t'] > 8.5:
                # Requires Remorque Tandem in addition to Crane
                equipment.append(['Crane', 'Remorque Tandem'])
            else:
                # Only Crane is needed
                equipment.append(['Crane'])
        else:
            # For Waste and Enrobé, select equipment based on Access Difficulty
            possible_equipment = client_needs[row['Need']]
            if row['Access_Difficulty'] == 'Yes':
                # Prefer trucks with Difficult Access capability
                equipment.append([random.choice(['Benne 10 m³', 'Benne 20 m³', 'Pont 13 m³'])])
            else:
                equipment.append([random.choice(possible_equipment)])
    
    clients_df['Required_Equipment'] = equipment
    
    return clients_df


In [37]:
def map_time_windows(clients_df):
    time_window_map = {
        'Morning': ('08:00', '12:00'),
        'Afternoon': ('13:00', '17:00'),
        'All Day': ('08:00', '17:00')
    }
    
    clients_df['Time_Window_Start'] = clients_df['Time_Window_Label'].map(lambda x: time_window_map[x][0])
    clients_df['Time_Window_End'] = clients_df['Time_Window_Label'].map(lambda x: time_window_map[x][1])
    
    return clients_df


In [38]:
# Define widgets
num_clients_widget = widgets.IntSlider(value=50, min=10, max=200, step=10,
                                       description='Number of Clients:', continuous_update=False)

waste_ratio_widget = widgets.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.05,
                                        description='Waste Ratio:', continuous_update=False)

enrobe_ratio_widget = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05,
                                         description='Enrobé Ratio:', continuous_update=False)

crane_ratio_widget = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.05,
                                        description='Crane Ratio:', continuous_update=False)

remorque_tandem_ratio_widget = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05,
                                                   description='Remorque Tandem %:', continuous_update=False)

difficult_access_ratio_widget = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.05,
                                                   description='Difficult Access %:', continuous_update=False)

@interact(num_clients=num_clients_widget,
          waste_ratio=waste_ratio_widget,
          enrobe_ratio=enrobe_ratio_widget,
          crane_ratio=crane_ratio_widget,
          remorque_tandem_ratio=remorque_tandem_ratio_widget,
          difficult_access_ratio=difficult_access_ratio_widget)
def generate_data(num_clients, waste_ratio, enrobe_ratio, crane_ratio, remorque_tandem_ratio, difficult_access_ratio):
    # Validate that ratios sum to 1 or less
    total_ratio = waste_ratio + enrobe_ratio + crane_ratio
    if total_ratio > 1.0:
        print("Error: The sum of waste, enrobé, and crane ratios should not exceed 1.0.")
        return
    
    # Generate clients
    clients = generate_clients(
        num_clients=num_clients,
        waste_ratio=waste_ratio,
        enrobe_ratio=enrobe_ratio,
        crane_ratio=crane_ratio,
        difficult_access_ratio=difficult_access_ratio,
        remorque_tandem_ratio=remorque_tandem_ratio
    )
    
    # Map time windows
    clients = map_time_windows(clients)
    
    # Display the first few rows of the generated data
    display(clients.head())
    
    # Store the generated clients in a global variable for later use
    global generated_clients
    generated_clients = clients



interactive(children=(IntSlider(value=50, continuous_update=False, description='Number of Clients:', max=200, …

In [39]:
def visualize_clients(clients_df):
    # Initialize Folium map centered around Paris
    paris_map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)
    
    # Define color and icon mapping
    color_map = {
        'Waste': 'blue',
        'Enrobé': 'green',
        'Crane Service': 'red'
    }
    
    # Define icon colors and types based on equipment
    def get_icon(equipment_list):
        if 'Remorque Tandem' in equipment_list:
            return folium.Icon(color='orange', icon='truck', prefix='fa')
        elif 'Crane' in equipment_list:
            return folium.Icon(color='red', icon='cloud', prefix='fa')
        elif 'Thermos' in equipment_list:
            return folium.Icon(color='green', icon='cog', prefix='fa')
        elif any(eq in ['Benne 10 m³', 'Benne 20 m³', 'Pont 13 m³'] for eq in equipment_list):
            return folium.Icon(color='blue', icon='circle', prefix='fa')
        else:
            return folium.Icon(color='gray', icon='info-sign', prefix='fa')
    
    marker_cluster = MarkerCluster().add_to(paris_map)
    
    for idx, row in clients_df.iterrows():
        popup_text = (f"Client ID: {row['Client_ID']}<br>"
                      f"Need: {row['Need']}<br>"
                      f"Volume: {row['Volume_kg']} kg<br>"
                      f"Access Difficulty: {row['Access_Difficulty']}<br>"
                      f"Time Window: {row['Time_Window_Start']} - {row['Time_Window_End']}<br>"
                      f"Required Equipment: {', '.join(row['Required_Equipment'])}")
        
        icon = get_icon(row['Required_Equipment'])
        
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=popup_text,
            icon=icon
        ).add_to(marker_cluster)
    
    display(paris_map)


In [40]:
visualize_button = widgets.Button(description="Visualize Clients on Map", button_style='success')

output_map = widgets.Output()

def on_button_click(b):
    with output_map:
        clear_output()
        if 'generated_clients' in globals():
            visualize_clients(generated_clients)
        else:
            print("Please generate the data first.")

visualize_button.on_click(on_button_click)

display(visualize_button, output_map)


Button(button_style='success', description='Visualize Clients on Map', style=ButtonStyle())

Output()

In [41]:
export_button = widgets.Button(description="Export to JSON", button_style='warning')

output_export = widgets.Output()

def on_export_click(b):
    with output_export:
        clear_output()
        if 'generated_clients' in globals():
            # Reorder columns for clarity
            export_df = generated_clients.copy()
            
            export_df = export_df[[
                'Client_ID',
                'Need',
                'Volume_kg',
                'Access_Difficulty',
                'Time_Window_Label',
                'Time_Window_Start',
                'Time_Window_End',
                'Latitude',
                'Longitude',
                'Required_Equipment'
            ]]
            
            # Export to JSON
            filename = f"fake_clients_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            export_df.to_json(filename, orient='records', lines=False, indent=4)
            print(f"Data exported to {filename}")
        else:
            print("Please generate the data first.")

export_button.on_click(on_export_click)

display(export_button, output_export)


Button(button_style='warning', description='Export to JSON', style=ButtonStyle())

Output()